In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
# ------------------------------------------define GPU id/s to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)
# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# this should import glob, os, and many other standard libs
# local imports
from src.utils.Notebook_imports import *
from src.utils.Utils_io import Console_and_file_logger, init_config

# import external libs
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import cv2
import pandas as pd

EXPERIMENT = 'cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8'
#EXPERIMENT = 'baseline_label_transpose_smooth05/36_5_BiLSTM32_NoBn_conv5_size1_CCE_NOphaseaug_NOaug_b8'
# EXPERIMENT = 'mased_scores/36_5_BiLSTM32_NoBn_conv5_size1_CCE_NOphaseaug_shift_rotate_reflectbordersgridaug'
timestemp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")) # ad a timestep to each project to make repeated experiments unique

EXPERIMENTS_ROOT = 'exp/'
EXP_PATH = os.path.join(EXPERIMENTS_ROOT, EXPERIMENT, timestemp)
MODEL_PATH = os.path.join(EXP_PATH, 'model', )
TENSORBOARD_PATH = os.path.join(EXP_PATH, 'tensorboard_logs')
CONFIG_PATH = os.path.join(EXP_PATH,'config')
HISTORY_PATH = os.path.join(EXP_PATH, 'history')
ensure_dir(MODEL_PATH)
ensure_dir(TENSORBOARD_PATH)
ensure_dir(CONFIG_PATH)
ensure_dir(HISTORY_PATH)

# define the data paths and fold 
# first to the 4D Nrrd files, 
# second to a dataframe with a mapping of the Fold-number
# Finally the path to the metadata
DATA_PATH_SAX = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/'
DF_FOLDS = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/df_kfold.csv'
DF_META = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase'
FOLD = 0

# General params
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 8 # 32, 64, 24, 16, 1 for 3D use: 4
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
EPOCHS = 50

DIM = [8, 64, 64] # network input shape for spacing of 3, (z,y,x)
T_SHAPE = 36
SPACING = [8, 3, 3] # if resample, resample to this spacing, (z,y,x)

# Model params
DEPTH = 4 # depth of the encoder
FILTERS = 32 # initial number of filters, will be doubled after each downsampling block
M_POOL = [1, 2, 2]# size of max-pooling used for downsampling and upsampling
F_SIZE = [3, 3, 3] # conv filter size
BN_FIRST = False # decide if batch normalisation between conv and activation or afterwards
BATCH_NORMALISATION = True # apply BN or not
PAD = 'same' # padding strategy of the conv layers
KERNEL_INIT = 'he_normal' # conv weight initialisation
OPTIMIZER = 'adam' # Adam, Adagrad, RMSprop, Adadelta,  # https://keras.io/optimizers/
ACTIVATION = 'relu' # tf.keras.layers.LeakyReLU(), relu or any other non linear activation function
LEARNING_RATE = 1e-4 # start with a huge lr to converge fast
REDUCE_LR_ON_PLAEAU_PATIENCE = 5
DECAY_FACTOR = 0.7 # Define a learning rate decay for the ReduceLROnPlateau callback
MIN_LR = 1e-12 # minimal lr, smaller lr does not improve the model
DROPOUT_min = 0.3 # lower dropout at the shallow layers
DROPOUT_max = 0.5 # higher dropout at the deep layers

# Callback params
MONITOR_FUNCTION = 'loss'
MONITOR_MODE = 'min'
SAVE_MODEL_FUNCTION = 'loss'
SAVE_MODEL_MODE = 'min'
MODEL_PATIENCE = 20
SAVE_LEARNING_PROGRESS_AS_TF = True

# Generator and Augmentation params
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = True # a compose of 2D augmentation (grid distortion, 90degree rotation, brightness and shift)
AUGMENT_PROB = 0.8
AUGMENT_PHASES = False
AUGMENT_PHASES_RANGE = (-18,18)
REPEAT_ONEHOT = True
SHUFFLE = True
RESAMPLE = True
SCALER = 'MinMax' # MinMax, Standard or Robust
# We define 5 target phases and a background phase for the pad/empty volumes 
PHASES = len(['ED#', 'MS#', 'ES#', 'PF#', 'MD#']) # skipped 'pad backround manually added', due to repeating
TARGET_SMOOTHING = True
SMOOTHING_KERNEL_SIZE = 12
SMOOTHING_LOWER_BORDER = 1
SMOOTHING_UPPER_BORDER = 5
SMOOTHING_WEIGHT_CORRECT = 10
config = init_config(config=locals(), save=True)


search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models
['/gpu:0', '/gpu:1']


In [ ]:
from src.models.train_model import train_fold

folds = [i for i in range(0, 4, 1)]

for f in folds:
    info('starting fold: {}'.format(f))
    config_ = config.copy()
    config_['FOLD'] = f
    train_fold(config_)

DEBUG:root:Creating directory /mnt/ssd/git/dynamic-cmr-models/logs/cv_baseline_NOtaug
2021-03-03 19:42:53,420 INFO -------------------- Start --------------------
2021-03-03 19:42:53,421 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-03-03 19:42:53,421 INFO Log file: ./logs/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0.log
2021-03-03 19:42:53,421 INFO Log level for console: INFO
2021-03-03 19:42:53,422 INFO Is built with tensorflow: True
2021-03-03 19:42:53,479 INFO Visible devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2021-03-03 19:42:54,117 INFO Local

['/gpu:0', '/gpu:1']
{'GPU_IDS': '0,1', 'GPUS': ['/gpu:0', '/gpu:1'], 'SEED': 42, 'EXPERIMENT': 'cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0', 'EXPERIMENTS_ROOT': 'exp/', 'EXP_PATH': 'exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42', 'MODEL_PATH': 'exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/model', 'TENSORBOARD_PATH': 'exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/tensorboard_logs', 'CONFIG_PATH': 'exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/config', 'HISTORY_PATH': 'exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/history', 'DATA_PATH_SAX': '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/', 'DF_FOLDS': '/mnt/ssd/data/gcn/02_importe

2021-03-03 19:42:54,119 INFO no files found, try to load with clean.nrrd/mask.nrrd pattern
2021-03-03 19:42:54,126 INFO Found 278 images/masks in /mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAX/
2021-03-03 19:42:54,126 INFO Patients train: 209
2021-03-03 19:42:54,137 INFO Selected 209 of 278 files with 209 of 279 patients for training fold 0
2021-03-03 19:42:54,138 INFO SAX train CMR: 209, SAX train masks: 209
2021-03-03 19:42:54,138 INFO SAX val CMR: 69, SAX val masks: 69
2021-03-03 19:42:54,151 INFO Check if we find the patient ID and phase mapping for all: 278 files.
2021-03-03 19:42:54,426 INFO Done!
2021-03-03 19:42:54,427 INFO Create DataGenerator
2021-03-03 19:42:54,428 INFO Datagenerator created with: 
 shape: [8, 64, 64]
 spacing: [8, 3, 3]
 batchsize: 8
 Scaler: MinMax
 Images: 209 
 Augment: True 
 Thread workers: 8
2021-03-03 19:42:54,428 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-03-03 19:42:54,442 INFO Smoothing kernel: 
[ 1.   1.8  2.6  

Shape after the temporal encoder
(None, 36, 8, 4, 4, 512)
Shape after GAP
(None, 36, 512)
Shape after Bi-LSTM layer
(None, 36, 512)
Shape after final conv layer
(None, 36, 5)


2021-03-03 19:42:58,944 INFO feed 4 Tensorboard is ready


...


Epoch 00020: loss improved from 0.42283 to 0.41846, saving model to exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/model/model.h5
26/26 [==============================] - 34s 1s/step - loss: 0.4185 - mse_wrapper: 0.4185 - ca_wrapper: 0.8303 - meandiff: 3.7548 - val_loss: 0.6766 - val_mse_wrapper: 0.6766 - val_ca_wrapper: 0.7973 - val_meandiff: 4.8281 - lr: 1.0000e-04
Epoch 21/50
26/26 [==============================] - ETA: 0s - loss: 0.4231 - mse_wrapper: 0.4231 - ca_wrapper: 0.8228 - meandiff: 3.7548
Epoch 00021: loss did not improve from 0.41846
26/26 [==============================] - 33s 1s/step - loss: 0.4231 - mse_wrapper: 0.4231 - ca_wrapper: 0.8228 - meandiff: 3.7548 - val_loss: 0.7313 - val_mse_wrapper: 0.7313 - val_ca_wrapper: 0.7613 - val_meandiff: 4.9375 - lr: 1.0000e-04
Epoch 22/50
26/26 [==============================] - ETA: 0s - loss: 0.4137 - mse_wrapper: 0.4137 - ca_wrapper: 0.8299 - meandiff: 3.8221

2021-03-03 19:56:17,832 INFO (4, 2, 36, 5)
2021-03-03 19:56:18,390 INFO (4, 2, 36, 5)



Epoch 00022: loss improved from 0.41846 to 0.41375, saving model to exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/model/model.h5
26/26 [==============================] - 35s 1s/step - loss: 0.4137 - mse_wrapper: 0.4137 - ca_wrapper: 0.8299 - meandiff: 3.8221 - val_loss: 0.6404 - val_mse_wrapper: 0.6404 - val_ca_wrapper: 0.8186 - val_meandiff: 4.4844 - lr: 1.0000e-04
Epoch 23/50
26/26 [==============================] - ETA: 0s - loss: 0.4123 - mse_wrapper: 0.4123 - ca_wrapper: 0.8341 - meandiff: 3.9087
Epoch 00023: loss improved from 0.41375 to 0.41228, saving model to exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/model/model.h5
26/26 [==============================] - 33s 1s/step - loss: 0.4123 - mse_wrapper: 0.4123 - ca_wrapper: 0.8341 - meandiff: 3.9087 - val_loss: 0.6465 - val_mse_wrapper: 0.6465 - val_ca_wrapper: 0.8047 - val_meandiff: 4.2812 - lr: 1.0000e-

2021-03-03 19:57:28,359 INFO (4, 2, 36, 5)
2021-03-03 19:57:28,891 INFO (4, 2, 36, 5)



Epoch 00024: loss did not improve from 0.41228
26/26 [==============================] - 34s 1s/step - loss: 0.4141 - mse_wrapper: 0.4141 - ca_wrapper: 0.8221 - meandiff: 3.9519 - val_loss: 0.9351 - val_mse_wrapper: 0.9351 - val_ca_wrapper: 0.7374 - val_meandiff: 4.4531 - lr: 1.0000e-04
Epoch 25/50
26/26 [==============================] - ETA: 0s - loss: 0.4071 - mse_wrapper: 0.4071 - ca_wrapper: 0.8301 - meandiff: 3.8510
Epoch 00025: loss improved from 0.41228 to 0.40706, saving model to exp/cv_baseline_NOtaug/8_64_64__8_3_3_4tenc_conv1_MSE_NOnorm_augshiftrot_NOtaug_3_batch8_f0/2021-03-03_19_42/model/model.h5
26/26 [==============================] - 33s 1s/step - loss: 0.4071 - mse_wrapper: 0.4071 - ca_wrapper: 0.8301 - meandiff: 3.8510 - val_loss: 0.7152 - val_mse_wrapper: 0.7152 - val_ca_wrapper: 0.8108 - val_meandiff: 4.3281 - lr: 1.0000e-04
Epoch 26/50
19/26 [====================>.........] - ETA: 7s - loss: 0.4040 - mse_wrapper: 0.4040 - ca_wrapper: 0.8353 - meandiff: 3.8026